## Imports

In [10]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from collections import defaultdict

## Settings

In [11]:
src_folder = '/home/drevital/obstacles_classification_datasets/obstacle_classification_RGB_data'
annotated_folder = '/home/drevital/obstacles_classification_datasets/rgb_6/annotated'
in_folders = ['/home/drevital/obstacles_classification_datasets/rgb_6/train']
out_folders = ['/home/drevital/obstacles_classification_datasets/7_channels/train']
sites = ['_'.join(s.split('_')[:-2]) for s in os.listdir(src_folder)]

In [12]:
sites

['musashi_office', 'koki_factory', 'israel', 'new_factory']

In [13]:
site_thresholds = {'israel': 55, 'new_factory': 50, 'musashi_office': 40, 'koki_factory': 40}
default_threshold = 50

## Make dictionary for the image names of each site

In [14]:
site_images = defaultdict(list)

for site in sites:
    site_folder = os.path.join(src_folder, site + '_rgb_data','all_data')
    class_folders = os.listdir(site_folder)
    for cls in class_folders:
        site_images[site] += [f for f in os.listdir(os.path.join(site_folder,cls))]

## List images not found in any site

In [15]:
class_folders = ['no_obstacle', 'obstacle']

for class_folder in class_folders:
    annotated = os.listdir(os.path.join(annotated_folder, class_folder))
    for a in annotated:
        # alt_name takes into account the same name with ignoring one _ at the end
        alt_name = '.'.join(a.split('.')[:-1])[:-1] + '.jpg'
        found_states = [a in site_images[site] for site in sites]
        found = any(found_states)
        alt_found = any([alt_name in site_images[site] for site in sites])
        found = found or alt_found
        if not found:
            print(f'{class_folder}: {a}')

## A funciton to find the source site of a given image

In [16]:
def find_site_and_threshold(im_name):
    found_states = [im_name in site_images[site] for site in sites]
    
    if any(found_states):
        site = sites[np.argmax(found_states)]
        threshold = site_thresholds[site]
    else:
        site = 'unknown'
        threshold = default_threshold
        
    return site, threshold

## A function to generate <ref, current, mask> triplet from <ref, current> pair

In [17]:
def triplet_image(pair, threshold):
    w = pair.shape[1]
    ref = pair[:, :w//2]
    current = pair[:, w//2:]
    diff = cv2.absdiff(current, ref)
    agg_rgb = np.stack((diff[:, :, 0], diff[:, :, 1], diff[:, :, 2])).max(0)
    _, mask = cv2.threshold(agg_rgb, threshold, 255, cv2.THRESH_BINARY)
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR) 
    
    return cv2.hconcat([ref, current, mask])

## Loop over in_folders, create <ref, current, mask> images and write the to corresponding out_folders

In [ ]:
class_names = ['no_obstacle', 'obstacle']

for i, in_folder in enumerate(in_folders):
    for class_name in class_names:
        class_path = os.path.join(in_folder, class_name)
        im_names = os.listdir(class_path)
        for im_name in im_names:
            im_path = os.path.join(class_path, im_name)
            pair = cv2.imread(im_path)
            site, threshold = find_site_and_threshold(im_name)
            triplet = triplet_image(pair, threshold)
            out_im_name = '.'.join(im_name.split('.')[:-1]) + f'_{site}_.jpg'
            out_path = os.path.join(out_folders[i], class_name, out_im_name)
            cv2.imwrite(out_path, triplet)